# FAST: Feedforward-Augmented Sentence Transformers

## Imports & Setup

In [1]:
# REQUIRED IMPORTS & SETUP

import os
import pickle
import numpy as np 
import pandas as pd 
import warnings
import itertools
from tqdm import tqdm

import torch
from sentence_transformers import SentenceTransformer
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score

from utils.feed_forward import FeedForward
from utils.cls import extract_cls_embeddings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Set Device ##########################################################
device_name = "cpu"  # default device is CPU
if torch.cuda.is_available():
    device_name = "cuda:0"  # CUDA for NVIDIA GPU
elif torch.backends.mps.is_available():
    device_name = torch.device("mps")  # Metal Performance Shaders for Apple M-series GPU
device = torch.device(device_name)
#######################################################################

# Load Models
Load models from HuggingFace and send to GPU device

In [2]:
mpnetv2 = SentenceTransformer("all-mpnet-base-v2").to(device)

## Load Data

In [3]:
data = load_dataset("glue", "mrpc")
data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

# Encode Sentence Embeddings

Generate sentence embeddings using sentence transformer model.

In [4]:
# for QNLI
# X_train = np.concatenate((mpnetv2.encode(data["train"]["question"]), mpnetv2.encode(data["train"]["sentence"])), axis=1)
# X_val = np.concatenate((mpnetv2.encode(data["validation"]["question"]), mpnetv2.encode(data["validation"]["sentence"])), axis=1)
# X_test = np.concatenate((mpnetv2.encode(data["test"]["question"]), mpnetv2.encode(data["test"]["sentence"])), axis=1)

# for MRPC
X_train = np.concatenate((mpnetv2.encode(data["train"]["sentence1"]), mpnetv2.encode(data["train"]["sentence2"])), axis=1)
X_val = np.concatenate((mpnetv2.encode(data["validation"]["sentence1"]), mpnetv2.encode(data["validation"]["sentence2"])), axis=1)
X_test = np.concatenate((mpnetv2.encode(data["test"]["sentence1"]), mpnetv2.encode(data["test"]["sentence2"])), axis=1)

Y_train = np.array(data["train"]["label"])
Y_val = np.array(data["validation"]["label"])
Y_test = np.array(data["test"]["label"])

In [21]:
X_train = mpnetv2.encode(data["train"]["sentence"])
X_val = mpnetv2.encode(data["validation"]["sentence"])
X_test = mpnetv2.encode(data["test"]["sentence"])

Y_train = np.array(data["train"]["label"])
Y_val = np.array(data["validation"]["label"])
Y_test = np.array(data["test"]["label"])

Save encodings and labels to disk for reuse. This is done because encoding embeddings takes a significant time but the encodings do not change throughout training, so we can cache it.

In [41]:
with open('./output/X_train.pt', 'wb') as X_train_file:
    torch.save(X_train, X_train_file)
with open('./output/X_val.pt', 'wb') as X_val_file:
    torch.save(X_val, X_val_file)
with open('./output/X_test.pt', 'wb') as X_test_file:
    torch.save(X_test, X_test_file)
with open('./output/Y_train.npy', 'wb') as Y_train_file:
    np.save(Y_train_file, Y_train)
with open('./output/Y_val.npy', 'wb') as Y_val_file:
    np.save(Y_val_file, Y_val)
with open('./output/Y_test.npy', 'wb') as Y_test_file:
    np.save(Y_test_file, Y_test)

Load saved encodings and labels from disk, if previously saved.

In [42]:
with open('./output/X_train.pt', 'rb') as X_train_file:
    X_train = torch.load(X_train_file)
with open('./output/X_val.pt', 'rb') as X_val_file:
    X_val = torch.load(X_val_file)
with open('./output/X_test.pt', 'rb') as X_test_file:
    X_test = torch.load(X_test_file)
with open('./output/Y_train.npy', 'rb') as Y_train_file:
    Y_train = np.load(Y_train_file)
with open('./output/Y_val.npy', 'rb') as Y_val_file:
    Y_val = np.load(Y_val_file)
with open('./output/Y_test.npy', 'rb') as Y_test_file:
    Y_test = np.load(Y_test_file)

## Define Hyperparameters
Defining hyperparameter grid for grid search

In [6]:
param_grid = {
    'num_epochs': [100],
    'batch_size': [32, 128, 512],
    'learning_rate': [1e-2, 1e-3, 1e-4, 1e-5],
    'category': ['C'],
    'norm': [False],
    'input_size': [768*2],
    'layer_size': [768*2],
    'num_layers': [1, 2, 3],
    'weight_decay':[1e-2, 1e-3, 1e-4, 1e-5],
    'patience': [3],
    'min_delta': [0],
    'device': [device_name]
}

# Create a list of all combinations of hyperparameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
print(f"{len(all_params)} hyperparameter combos")

best_params = None
highest_val_accuracy = 0

144 hyperparameter combos


# Training Loop for Hyperparameter Grid Search

In [7]:
verbose = True

# Iterate over all combinations of hyperparameters
bar = tqdm(all_params)
for params in bar:
    if verbose: print("\nTraining with parameters:\n", params)
    # Initialize the model with current set of hyperparameters
    feed_forward = FeedForward(**params)
    
    epoch, val_loss, val_accuracy, val_f1, val_mcc = feed_forward.fit(X_train, Y_train, X_val, Y_val)

    if verbose:
        print("Early stopped on epoch:", epoch)
        print("Validation accuracy:", val_accuracy)
        print("Validation f1-score:", val_f1)
        print("Validation MCC     :", val_mcc)
    
    # Save the parameters if they provide a better accuracy
    if val_accuracy > highest_val_accuracy:
        highest_val_accuracy = val_accuracy
        best_params = params

    bar.set_description(f"Current best val acc: {highest_val_accuracy:.5f}")

# Print the best parameters
print("\nBest Parameters:", best_params)
print("Highest Validation Accuracy:", highest_val_accuracy)

  0%|          | 0/144 [00:00<?, ?it/s]


Training with parameters:
 {'num_epochs': 100, 'batch_size': 32, 'learning_rate': 0.01, 'category': 'C', 'norm': False, 'input_size': 1536, 'layer_size': 1536, 'num_layers': 1, 'weight_decay': 0.01, 'patience': 3, 'min_delta': 0, 'device': 'cuda:0'}


Current best val acc: 0.62745:   1%|          | 1/144 [00:00<01:28,  1.62it/s]

Early stopped on epoch: 4
Validation accuracy: 0.6274509803921569
Validation f1-score: 0.742372881355932
Validation MCC     : 0.07839013055430459

Training with parameters:
 {'num_epochs': 100, 'batch_size': 32, 'learning_rate': 0.01, 'category': 'C', 'norm': False, 'input_size': 1536, 'layer_size': 1536, 'num_layers': 1, 'weight_decay': 0.001, 'patience': 3, 'min_delta': 0, 'device': 'cuda:0'}


Current best val acc: 0.65686:   1%|▏         | 2/144 [00:01<01:24,  1.68it/s]

...

Early stopped on epoch: 53
Validation accuracy: 0.7009803921568627
Validation f1-score: 0.8195266272189349
Validation MCC     : 0.179710163362961

Training with parameters:
 {'num_epochs': 100, 'batch_size': 512, 'learning_rate': 1e-05, 'category': 'C', 'norm': False, 'input_size': 1536, 'layer_size': 1536, 'num_layers': 3, 'weight_decay': 0.0001, 'patience': 3, 'min_delta': 0, 'device': 'cuda:0'}


Current best val acc: 0.71078:  99%|█████████▉| 143/144 [02:55<00:02,  2.78s/it]

Early stopped on epoch: 57
Validation accuracy: 0.7058823529411765
Validation f1-score: 0.8219584569732937
Validation MCC     : 0.20677292192951885

Training with parameters:
 {'num_epochs': 100, 'batch_size': 512, 'learning_rate': 1e-05, 'category': 'C', 'norm': False, 'input_size': 1536, 'layer_size': 1536, 'num_layers': 3, 'weight_decay': 1e-05, 'patience': 3, 'min_delta': 0, 'device': 'cuda:0'}


Current best val acc: 0.71078: 100%|██████████| 144/144 [02:57<00:00,  1.24s/it]

Early stopped on epoch: 50
Validation accuracy: 0.6985294117647058
Validation f1-score: 0.8183161004431315
Validation MCC     : 0.16493341660730046

Best Parameters: {'num_epochs': 100, 'batch_size': 512, 'learning_rate': 0.0001, 'category': 'C', 'norm': False, 'input_size': 1536, 'layer_size': 1536, 'num_layers': 1, 'weight_decay': 0.0001, 'patience': 3, 'min_delta': 0, 'device': 'cuda:0'}
Highest Validation Accuracy: 0.7107843137254902


# Evaluate Best Hyperparameters

In [48]:
best_feed_forward = FeedForward(**best_params)

X = np.concatenate((X_train, X_val), axis=0)
Y = np.concatenate((Y_train, Y_val), axis=0)

best_feed_forward.fit(X, Y)

preds = np.argmax(best_feed_forward.predict_proba(X_test), axis=1)
print(preds.shape)


df = pd.DataFrame({
    'index': range(len(preds)),
    'prediction': preds
})

# Write the DataFrame to a .tsv file, without the header and index
df.to_csv('QNLI.tsv', sep='\t', index=False, header=True)

(5463,)
